# Proyecto MusicStream – Año 2016

Extracción de datos desde Spotify y Last.fm usando funciones separadas.
Lógica clara, modular y fácil de explicar.

## 1. Importaciones y configuración

In [1]:
# Extracción de datos 
# --- 1. SISTEMA Y SEGURIDAD ---
# Gestiona variables de entorno y control de pausas
from dotenv import load_dotenv       # Carga el archivo .env con claves secretas
load_dotenv()                        # Inyecta las claves como variables de entorno
import os                              # Accede a las claves con os.environ.get
import time                            # Permite pausas controladas para no saturar APIs

# --- 2. EXTRACCIÓN Y MANEJO DE APIS ---
# Comunicación y autenticación con APIs de música
import requests                        # Peticiones HTTP genéricas (ej. Last.fm)
import spotipy                          # Wrapper para la API de Spotify
from spotipy.oauth2 import SpotifyClientCredentials  # OAuth seguro de Spotify
from spotipy.exceptions import SpotifyException      # Manejo de errores de Spotify (ej. 429)

# --- 3. ESTRUCTURA Y TRANSFORMACIÓN DE DATOS ---
import pandas as pd                     # DataFrames para limpiar, unir y manipular datos

# --- 4. NORMALIZACIÓN Y UTILIDADES ---
from urllib.parse import quote           # Codifica URLs para nombres con espacios/acentos


In [2]:
import sys
sys.executable


'c:\\Users\\bruno\\anaconda3\\python.exe'

In [3]:
!{sys.executable} -m pip install mysql-connector-python


In [4]:
import mysql.connector


In [5]:
import mysql.connector
print(mysql.connector.__version__)


9.5.0


In [6]:
# Conexión a sql
import mysql.connector
from mysql.connector import Error # Manejo de errores específicos de MySQL

# --- Otras Utilidades ---
import numpy as np # Necesario para manejar np.nan (valores nulos) al insertar DataFrames


## 2. Credenciales: Spotify

In [7]:
# --- COMPROBACIÓN Y CONEXIÓN CON SPOTIFY ---
# Recupera las credenciales del archivo .env
client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

# Verifica que ambas credenciales estén disponibles
if not client_id or not client_secret:
    print("⚠️ ADVERTENCIA: Las credenciales de Spotify no están cargadas. Revisa tu archivo .env.")
else:
    # Configura la autenticación segura con Spotify usando OAuth de aplicación
    mis_credenciales = SpotifyClientCredentials(
        client_id=client_id,
        client_secret=client_secret
    )
    # Inicializa el cliente de Spotify con las credenciales
    spotify = spotipy.Spotify(auth_manager=mis_credenciales)
    print("✅ Conexión con Spotify inicializada.")


✅ Conexión con Spotify inicializada.


In [8]:
# --- PRUEBA DE CONEXIÓN CON SPOTIFY ---
# Objetivo: comprobar que el cliente de Spotify funciona y devuelve datos válidos.

# Realizamos una búsqueda de artistas por género (genre solo funciona con artistas)
prueba = spotify.search(
    q='genre:latin',   # Buscamos artistas del género "latin"
    type='artist',     # Tipo de búsqueda: artistas
    limit=1            # Solo 1 resultado, suficiente para probar
)

# Extraemos el primer artista de los resultados
artista = prueba['artists']['items'][0]

# Mostramos el nombre del artista para confirmar que la conexión y la búsqueda funcionan
print("✅ Conexión exitosa con Spotify")
print("🎵 Artista de prueba encontrado:", artista['name'])

✅ Conexión exitosa con Spotify
🎵 Artista de prueba encontrado: Bad Bunny


In [9]:
# --- COMPROBACIÓN DE LA CLAVE DE LAST.FM ---
# Recupera la API key desde el archivo .env
api_key_lastfm = os.getenv("API_KEY_LASTFM")

# Verifica que la clave exista antes de usarla
if not api_key_lastfm:
    print("⚠️ ADVERTENCIA: La clave de Last.fm no está cargada. Revisa tu archivo .env.")
else:
    # Confirmamos que la clave está disponible para futuras peticiones
    print("✅ Conexión con Last.fm inicializada.")

✅ Conexión con Last.fm inicializada.


In [10]:
# --- PRUEBA DE CONEXIÓN BÁSICA CON LAST.FM ---
if not api_key_lastfm:
    print("❌ Clave API de Last.fm no definida.")
else:
    try:
        # Hacemos una petición mínima para verificar la conexión usando el método 'chart.getTopArtists'
        respuesta = requests.get(
            "http://ws.audioscrobbler.com/2.0/",
            params={
                'method': 'chart.gettopartists',
                'api_key': api_key_lastfm,
                'format': 'json',
                'limit': 1  # Solo necesitamos 1 artista para probar
            }
        )
        respuesta.raise_for_status()
        print("✅ Conexión exitosa con Last.fm")
    except requests.exceptions.RequestException as e:
        print(f"❌ Error al conectar con Last.fm: {e}")

✅ Conexión exitosa con Last.fm


## 3. Obtener información Spotify

In [ ]:
generos = ["classical", "latin", "jazz", "rock"]
año = 2018
todas_las_canciones = []
albumes_ya_vistos = set()  # Evita duplicados: un álbum puede aparecer al buscar varios artistas (ej. colaboraciones).
datos_artistas = {}
artistas_procesados = set()

print (f"Buscando canciones de {año}\n")

for genero in generos:
    print("Género:", genero)

    resultado_artistas = spotify.search(q="genre:" + genero, type="artist", limit=50)
    artistas = resultado_artistas["artists"]["items"]

    for artista in artistas:
        nombre_artista = artista["name"]

        #  Se buscan ámbumes del artista en el año informado
        busqueda = "artist:" + nombre_artista + " year:" + str(año)
        resultado_albumes = spotify.search(q=busqueda, type="album", limit=50)
        albumes = resultado_albumes["albums"]["items"]
        
        for album in albumes:
            id_album = album["id"]
            nombre_album = album["name"]
            
            # Igoramos álbumes ya vistos
            if id_album in albumes_ya_vistos:
                continue
            
            #se añaden los álmbumes al set
            albumes_ya_vistos.add(id_album)

            # Y cogemos Pedimos las canciones del álbum
            resultado_canciones = spotify.album_tracks(id_album)
            canciones = resultado_canciones["items"]

            for cancion in canciones:
                info = {
                    "nombre": cancion["name"],
                    "artista": nombre_artista,
                    "album": nombre_album,
                    "genero": genero,
                    "año": año
                }
                todas_las_canciones.append(info)

    print(f"Añadidas canciones de '{genero}'\n")
   

# Contamos las canciones de cada género una por una
for genero in generos:
    contador = 0
    for cancion in todas_las_canciones:
        if cancion["genero"] == genero:
            contador = contador + 1
    print(f"- {genero}: {contador} canciones")

print(f"Canciones añadidas: {len(todas_las_canciones)}")

todas_las_canciones_df = pd.DataFrame(todas_las_canciones)

Buscando canciones de 2018

Género: classical


Your application has reached a rate/request limit. Retry will occur after: 83030 s


In [ ]:
todas_las_canciones_df.to_csv("canciones_2018.csv", index=False)

In [11]:
#Last FM
if not api_key_lastfm:
    print("ERROR: La variable de entorno 'API_KEY_LASTFM' no está configurada.")
else:
    print("API Key de Last.fm cargada con éxito.")

API Key de Last.fm cargada con éxito.


## 4. Obtener información LAST.FM

In [14]:
# ────────────────────────────────────────────────
# FUNCIÓN PARA CONSULTAR INFO DE ARTISTA EN LAST.FM
# ────────────────────────────────────────────────
def busqueda_info_artista(artista, api_key):
    """
    Consulta resumen y oyentes de un artista en Last.fm
    """
    url = "http://ws.audioscrobbler.com/2.0/"
    params = {
        "method": "artist.getinfo",
        "artist": quote(artista),
        "api_key": api_key,
        "format": "json"
    }

    try:
        resp = requests.get(url, params=params, timeout=10)
        resp.raise_for_status()
        data = resp.json()

        if "artist" in data:
            bio = data["artist"].get("bio", {}).get("summary", "").split("<a href")[0].strip()
            listeners = int(data["artist"].get("stats", {}).get("listeners", 0))
            return {"artista": artista, "bio_resumen": bio, "listeners": listeners, "consulta_exitosa": True}
        else:
            return {"artista": artista, "consulta_exitosa": False, "error_lastfm": "No encontrado"}

    except Exception as e:
        return {"artista": artista, "consulta_exitosa": False, "error_lastfm": str(e)}

In [ ]:
# ────────────────────────────────────────────────
# GENERAR DATAFRAME DE ARTISTAS
# ────────────────────────────────────────────────
def generar_datos_lastfm(canciones_df, api_key):
    if not api_key:
        print("❌ ERROR: La clave de Last.fm no está configurada")
        return None

    artistas_unicos = canciones_df['artista'].unique()
    resultados = []

    # Bucle simple sobre cada artista
    for artista in artistas_unicos:
        print(f"Consultando {artista}...")
        info = busqueda_info_artista(artista, api_key)
        resultados.append(info)

    # Convertir a DataFrame
    df_lastfm = pd.DataFrame(resultados)
    return df_lastfm

# Supongamos que tienes tu DataFrame de Spotify
# todas_las_canciones_df

api_key_lastfm = os.getenv("API_KEY_LASTFM") # Aseguramos que se usa la clave cargada

df_lastfm = generar_datos_lastfm(todas_las_canciones_df, api_key_lastfm)

print(df_lastfm.head())

In [ ]:
datos_lastfm_df 

In [ ]:
datos_lastfm_df.to_csv("artistas_lastfm_2018.csv", index=False)

# 5. CREACIÓN DE LA BASE DE DATOS

In [18]:
# A. Conexión a MySQL
try:
    # Intentamos conectarnos al servidor MySQL
    cnx = mysql.connector.connect(
        host='127.0.0.1',    # Dirección del servidor MySQL (localhost)
        user='root',         # Usuario con el que nos conectamos
        password='AlumnaAdalab',  # Contraseña del usuario
        use_pure=True,       # use_pure=True garantiza compatibilidad con Python 3.12, evita problemas en el Kernel
    )
    print('Conexión exitosa')  # Confirmamos que la conexión fue correcta
except mysql.connector.Error as e:  # Captura errores relacionados con MySQL
    print('Error al conectar:', e)   # Muestra el error para diagnosticar el problema



Conexión exitosa


In [ ]:
# B. Creación de la base de datos

try:
    # Creamos un cursor: es el intermediario entre Python y MySQL, necesario para ejecutar consultas SQL
    mycursor = cnx.cursor()

    # Consulta SQL para crear la base de datos si no existe
    # "IF NOT EXISTS" evita errores si la base de datos ya está creada
    query_create_database = "CREATE DATABASE IF NOT EXISTS MusicStream_db"

    # Ejecutamos la consulta SQL para crear la base de datos
    mycursor.execute(query_create_database)

    # Seleccionamos explícitamente la DB para las operaciones futuras (Creación de tablas e inserción)
    mycursor.execute("USE MusicStream_db")

    print("Base de datos creada y seleccionada correctamente.")

# Captura errores relacionados con MySQL (por ejemplo, permisos insuficientes)
except mysql.connector.Error as e:  
    print("Ocurrió un error al crear la base de datos:", e)


NameError: name 'cnx' is not defined

In [ ]:
# C. Creación de las tablas

# ------------------ Tabla Canciones ------------------
query_canciones = '''CREATE TABLE IF NOT EXISTS Canciones(
    ID_Cancion INT AUTO_INCREMENT,
    Nombre TEXT NOT NULL,
    Artista VARCHAR(45),
    Album TEXT,
    Genero VARCHAR(30),
    Lanzamiento YEAR,
    PRIMARY KEY (ID_Cancion)
);'''

try:
    mycursor.execute(query_canciones)
    print("Tabla 'Canciones' creada correctamente (o ya existía).")
except mysql.connector.Error as e:
    print("Ocurrió un error al crear la tabla 'Canciones':", e)
    
# ------------------ Tabla Artistas ------------------
query_artistas = '''CREATE TABLE IF NOT EXISTS Artistas(
    ID_Artista INT AUTO_INCREMENT,
    Artista VARCHAR(70) UNIQUE NOT NULL,
    Info TEXT,
    Oyentes INT,
    PRIMARY KEY (ID_Artista)
);'''

try:
    mycursor.execute(query_artistas)
    print("Tabla 'Artistas' creada correctamente (o ya existía).")
except mysql.connector.Error as e:
    print("Ocurrió un error al crear la tabla 'Artistas':", e)

Tabla 'Canciones' creada correctamente (o ya existía).
Tabla 'Artistas' creada correctamente (o ya existía).


# 6. INSERCIÓN DE LA BASE DE DATOS

In [ ]:
# Inserción de datos en la tabla 'Canciones'

# Seleccionamos la base de datos (redundante si se hizo en la Celda B, pero seguro)
mycursor.execute("USE MusicStream_db")

# Consulta de inserción de canciones
query_insert_canciones = """
INSERT INTO Canciones (Nombre, Artista, Album, Genero, Lanzamiento) 
VALUES (%s, %s, %s, %s, %s)
"""

try:
    # 1. Preparar datos del DataFrame de Spotify
    # Reemplazamos valores nulos (NaN) por None
    df_corregido_SPOTIFY = todas_las_canciones_df.replace({np.nan: None, 'nan': None, 'NaN': None})
    
    # 2. Filtrado de Columnas: Seleccionamos solo las 5 columnas que coinciden con la tabla SQL
    # Las columnas del DataFrame son 'nombre', 'artista', 'album', 'genero', 'año'
    datos = df_corregido_SPOTIFY[['nombre', 'artista', 'album', 'genero', 'año']].values.tolist()

    # 3. Ejecutar la inserción masiva
    mycursor.executemany(query_insert_canciones, datos)
    
    print(f"✅ {mycursor.rowcount} registros de canciones insertados correctamente.")
    
    # 4. Commit: Usamos 'cnx' que es la variable de conexión definida
    cnx.commit()  # Guardamos los cambios
    

except Error as e:
    print("❌ Error al insertar los datos de Spotify:", e)
    # Rollback: Usamos 'cnx' en caso de error
    cnx.rollback()  # Revertimos la operación en caso de error

✅ 1079 registros de canciones insertados correctamente.


In [ ]:
cnx.close()